[Section 1](#section_1)

## Imports

In [540]:
import pandas as pd
import os
import numpy as np
from pyvis.network import Network
from neo4j import GraphDatabase
import re
from functools import reduce


## Functions 

### Find Each Table's primary key

In [541]:
def pk_finder(name, df):
    for i in range(len(df.columns)):
        col = df.iloc[:,i]
        #if(col.size == col.drop_duplicates().size):
        if(col.nunique() == col.size):
            pk[name] = df.columns[i] 
            break
        

### Find Each Table's foreign key


In [542]:
def fk_finder(name, df):
    for i in range(len(df.columns)):
        col_name = df.columns[i]
        col = df[col_name]
        for j in pk: #loops on the key j = table names
            if j != name:
                primary_key_column = All_dfs[j][(pk[j])]
                
                if primary_key_column.dtype == col.dtype:
                    status = col.isin(primary_key_column)
                    if status[status== False].size == 0:
                        fk[name][col_name] = j 
                        ref_in[j].add(name)
                        break

                elif (isinstance(col[0],list) and type(primary_key_column[0]) == type(col[0][0])):
                    status = col.explode().reset_index(drop=True).isin(primary_key_column)
                    if status[status== False].size == 0:
                        fk[name][col_name] = j 
                        ref_in[j].add(name)
                        break

In [543]:
# type(All_dfs["warehouses"].warehouse_id[0])

### Assign Tables to nodes or edges

In [544]:
def nodes_edges(fk):
    for f in fk:
        if len(fk[f]) == 2:
            inner_dict = list(fk[f])
            edges[f] = fk[f]
        elif len(fk[f]) == 1:
            properties[f] = fk[f]
        else:
            nodes[f] = All_dfs[f]
#         else:
#             if len(fk[f]) == 1:
#                 properties[f] = fk[f]
            
#             nodes[f] = All_dfs[f]
    # for f in fk:
    #     if((f not in edges) ):
    #         nodes[f] = All_dfs[f]
    #         #nodes.append(f)
        

### Create Nodes Table

In [545]:
def add_nodes(nodes, nodesTable):
    for n in range(len(nodes)):
        for index, row in nodes[list(nodes.keys())[n]].iterrows():
            column_names = list(nodes[list(nodes.keys())[n]].columns) #get column names
            att = {}
            for i in range(1, len(column_names)):
                att[column_names[i]] = nodes[list(nodes.keys())[n]].iloc[index, i]
            newRow = [{'Label': list(nodes.keys())[n] , 'ID': nodes[list(nodes.keys())[n]].iloc[index,0] , 'Attributes': att}]
            tmp = pd.DataFrame(newRow)
            nodesTable = pd.concat([nodesTable, tmp], ignore_index=True)
    return nodesTable


### Create Edges Table

In [546]:
def add_edges(edges,edgesTable,pk,fk,ref_in):
    for e in edges:
        df = All_dfs[e] #df of the cur edge
        for r in range(len(df)):
            label = list(edges[e].keys())
            from_id = nodesTable[(nodesTable["Label"] == edges[e][label[0]]) & (df.loc[r,label[0]] == nodesTable["ID"])]
            from_id = from_id.index[0]
            to_id = nodesTable[(nodesTable["Label"] == edges[e][label[1]]) & (df.loc[r,label[1]] == nodesTable["ID"])]
            to_id = to_id.index[0]
            pk_col = pk[e]
#             primary_key = df[pk_col].iloc[r]
            primary_key = df.loc[r,pk_col]
            att = convert_prop(ref_in,e,primary_key,fk)
            newRow = [{'From_Node_ID': from_id, 'To_Node_ID': to_id, 'order/service' : att}]
            tmp = pd.DataFrame(newRow)
            edgesTable = pd.concat([edgesTable, tmp], ignore_index=True)
    return edgesTable

### Properties of the Edge

In [547]:
def convert_prop(ref_in,edge_key, pk_value,fk):
    for referenced_table_name in ref_in[edge_key]:
        referenced_table = All_dfs[referenced_table_name]
        fk_of_ref_table = fk[referenced_table_name]
        for foreign_key in fk_of_ref_table:
            if fk_of_ref_table[foreign_key] == edge_key:
#                 referenced_table[referenced_table[foreign_key]== id]
                all_occurances_df = referenced_table[referenced_table[foreign_key]== pk_value].drop([foreign_key],axis=1)
                return all_occurances_df.to_dict('records')
                

### generate list of n colors

In [548]:
import random
get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
# get_colors(5) # sample return:  ['#8af5da', '#fbc08c', '#b741d0', '#e599f1', '#bbcb59', '#a2a6c0']

### Draw Pyvis graph

In [549]:
def draw_graph_pyvis(nodes_table,edges_table):

    nodes_name = [(x[1]+'_'+str(x[0])).capitalize() for x in zip(nodes_table.ID,nodes_table.Label)]
    indices = list(map(lambda x: int(x),list(nodes_table.index)))
    nodes_attributes = list(map(lambda x: str(x).replace(',',"\n"),list(nodes_table.Attributes)))
    nodes_tables = list(nodes_df.Label.unique())
    colors = get_colors(len(nodes_name))
    nodes_color = list(map(lambda x: colors[nodes_tables.index(x)],list(nodes_table.Label)))
    
    g = Network(width='100%')
#     g.barnes_hut()
    g.add_nodes(indices,
                title= nodes_attributes,
                label=nodes_name,
                color=nodes_color,
                )
    for e in range(len(edges_table)):
        src = int(edges_table.iloc[e]["From"])
        dst = int(edges_table.iloc[e]["To"])
        g.add_edge(src,dst,weight=5)

    print("Number of Nodes: ",len(g.get_nodes()))
    print("Number of Edges: ",len(g.get_edges()))
    

    g.show('test.html')




### Draw Graph Neo4j

<a id='Neo4jFunction'></a>

[Noe4jCallFunction](#Neo4jCall)

#### prepare satements

In [550]:
def draw_graph_Neo4j(nodes_table,edges_table):

    transaction_execution_commands = []
    # creating a statement for each node (row in node_table)
    for i, node in nodes_table.iterrows():
        label = node["Label"].capitalize()
        index = i
        IDs = node["ID"]
        attributes = node["Attributes"]
        neo4j_create_statement = Create_Statement(label,i,IDs,attributes)
        transaction_execution_commands.append(neo4j_create_statement)

    for i, edge in edges_table.iterrows():
        from_node_id = edge['From']
        to_node_id = edge['To']
        from_node_name = edge['From_Table']
        to_node_name = edge['To_Table']
        neo4j_create_statement = create_relation_statement(from_node_id,to_node_id,from_node_name,to_node_name)
        transaction_execution_commands.append(neo4j_create_statement)

     
    execute_transactions(transaction_execution_commands)
 

#### Connect and execute statements

In [551]:
def execute_transactions(transaction_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "password"))
    session = data_base_connection.session()    
    for i in transaction_execution_commands:
        session.run(i)

 

#### Create a "Create statement for nodes"

In [552]:
def Create_Statement(label,index,IDs,attributes):
#     att = destructure_dict(attributes)
#     return f"CREATE (x:{label} {'{'}name: {get_name(label,IDs)},index:{index},ID:{IDs} ,{destructure_dict(attributes)} {'}'})"
    att = (", " + destructure_dict(attributes)) if len(attributes) > 0 else ""
    return f"CREATE (x:{label} {'{'}name: {get_name(label,IDs)},index:{index},ID:{IDs} {att} {'}'})"


In [553]:
def get_name(label,ID):
    return '\"' + label.capitalize()+' '+str(ID)+'\"'

#### prepare attributes for create statement

In [554]:
def destructure_dict(dic):
    s = ""
    for key in dic:
        value = dic[key]
        if isinstance (value,str):
            value = "\"" + value +"\""
        s=s + re.sub("[^\w_]", '',str(key).replace(" ","_")) + ":" + str(value)+","
    return s[:-1]

In [555]:
def create_relation_statement(from_id, to_id,from_name,to_name):
        return f"Match (a:{from_name}),(b:{to_name}) WHERE a.index ={from_id} AND b.index = {to_id} CREATE (a) - [r:RELTYPE]->(b)"


In [556]:
def and_agg(series):
       return reduce(lambda x, y: x and y, series)

## Global Variables

In [557]:
# {Table name: df}
All_dfs = {}
# {Table name: Table's Primary key}
pk = {}
# {Table name: {Column Name: Referenced Table}}
fk = {}
# {Table name: set(Tables that references the table)}
ref_in = {}
# {Table name: df}
nodes = {}
# {Table name: {Column Name: Referenced Table}}
edges = {}
# _
properties = {}
# Nodes_df
nodesTable = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
# Edges_df
edgesTable = pd.DataFrame(columns = ['From_Node_ID', 'To_Node_ID', 'order/service'])
# Properties_df
propertiesTable = pd.DataFrame(columns = [['index','From', 'Label', 'Attributes']])
#Label = ['facilities'/ 'order' / 'service']


# Nodes
# Edges ['From_Node_ID', 'To_Node_ID']
# Properties ['Label', 'ID', 'Attributes']
# (^Nodes in graph)
# Relationship ["Relation_name","from","to"]


nodes_df = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
edges_df = pd.DataFrame(columns=['From','To','From_Table','To_Table'])


<a id='section_1'></a>
## Reading Data Set


 creating a dictionary where
 key: first word of the table name 
 value: df 

In [558]:
import os
path_of_the_directory = './DataSet/'
ext = ('.csv')
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        print(file) 
        temp = (file.replace("_"," ").replace("."," ").split(" ")[0].lower())
        All_dfs[temp] = pd.read_csv(path_of_the_directory+file)
    else:
        continue

InternalTransactions.csv
InternalShipments_data.csv
facilities_data.csv
ExternalShipments_data.csv
InternalServices_data.csv
warehouses_data.csv
InternalOrders_data.csv
customer_data.csv
Products_data.csv
Supplier_data.csv
ExternalTransactions.csv
ExternalOrders_data.csv
Manufacturing_data.csv
ExternalServices_data.csv
Retailer_data.csv


In [559]:
for table_nm in All_dfs:
    table = All_dfs[table_nm]
    for column_name in table.columns:
        column =  table[column_name]
        if column.dtype=='O' and isinstance(column[0],str):
            if  (isinstance(column[0],str) and column.apply(lambda x: (str(x).startswith('[') and str(x).endswith(']'))
                         or 
                         (str(x).startswith('(') and str(x).endswith(')'))).agg(and_agg)):
                
                s = column.apply(lambda x: x.strip("[](,)").split(','))
                if (s[0][0].isdigit()):
                    s = s.apply(lambda x: list(map(np.int64,x)))
                All_dfs[table_nm][column_name] = s
                


In [560]:
All_dfs["products"].warehouses.explode().isin(All_dfs["warehouses"].warehouse_id ).agg(and_agg)

True

In [561]:
# ware_house_id = All_dfs["warehouses"].warehouse_id 

# warehouse_product_isin_warehouse = All_dfs["products"].warehouses.explode().isin(ware_house_id)
# warehouse_product_not_in_warehouse_values = list(All_dfs["products"].warehouses.explode()[[not elem for elem in warehouse_product_isin_warehouse]].unique())
# print(len(warehouse_product_not_in_warehouse_values))
# warehouse_isin_warehouse_product =  ware_house_id.isin(All_dfs["products"].warehouses.explode())
# print(len( list(ware_house_id[[not elem for elem in warehouse_isin_warehouse_product]].unique())))
# warehouses_to_be_replaced = list(ware_house_id[[not elem for elem in warehouse_isin_warehouse_product]].unique())[:len(warehouse_product_not_in_warehouse_values)]

# All_dfs["warehouses"].warehouse_id  =  ware_house_id.replace(warehouses_to_be_replaced,warehouse_product_not_in_warehouse_values)

### Adding Missing Edges


In [562]:
initial_internalshipments_df = All_dfs["internalshipments"]
initial_internalshipments_df

,IntShip_id,listSuppIds,factoryIds,from_to_where
0,7281,90942,52160,SS
1,5897,36243,67361,SR
2,3477,87916,30692,SS
3,8796,90236,26533,SS
4,2360,89467,46954,SR
...,...,...,...,...
95,5797,11283,24646,SS
96,6673,19072,31857,SR
97,3632,29277,97142,SS
98,3089,72315,90349,SR


In [563]:
c = 9995
from_shipment = All_dfs["internalshipments"].listSuppIds
to_shipment = All_dfs["internalshipments"].factoryIds

for r in range(len(All_dfs["manufacturing"])):
    diff_supp = All_dfs["manufacturing"].loc[r,"Different_suppliers"]
    factory = All_dfs["manufacturing"].loc[r,"Factory_id"]
    for s in range(len(diff_supp)):
        
        supp = diff_supp[s]
        if supp == factory:
            print(factory)
        if len(All_dfs["internalshipments"].query(f"listSuppIds == {supp} and factoryIds == {factory}")) ==0:
            new_row = [{"IntShip_id":c, "listSuppIds":supp, "factoryIds": factory, "from_to_where": "SS"}] 
            df = pd.DataFrame(new_row)
            c = c+1
            All_dfs["internalshipments"] = pd.concat([All_dfs["internalshipments"], df], ignore_index=True)

#         if (supp not in from_shipmentfrom_shipment)  :
#                 if factory in All_dfs["supplier"].supp_id:
#                     i = "SS"

#         new_row = [{"IntShip_id":c, "listSuppIds":diff_supp[s], "factoryIds": factory, "from_to_where": i}] 
#         df = pd.DataFrame(new_row)
#         c = c+1
#         All_dfs["internalshipments"] = pd.concat([All_dfs["internalshipments"], df], ignore_index=True)
      

In [564]:
All_dfs["internalshipments"]

,IntShip_id,listSuppIds,factoryIds,from_to_where
0,7281,90942,52160,SS
1,5897,36243,67361,SR
2,3477,87916,30692,SS
3,8796,90236,26533,SS
4,2360,89467,46954,SR
...,...,...,...,...
583,10478,42179,46070,SS
584,10479,48467,46070,SS
585,10480,70468,18952,SS
586,10481,48580,18952,SS


In [565]:
All_dfs["internalorders"]

,IntOrders_id,IntShip_id,quantity,placed_when,actual_date,expected_date,cost,status
0,8605,2360,766,1993-05-08,1993-05-30 00:00:00,1993-06-01,38.90,Closed
1,6266,6224,438,1975-01-08,NaN,1975-02-01,7.95,Out For Delivery
2,3057,1388,771,2007-10-29,2007-11-08 00:00:00,2007-11-16,8.24,Closed
3,7340,9456,870,2000-07-24,2000-08-07 00:00:00,2000-08-03,17.14,Delayed
4,5318,6798,801,2021-01-18,2021-02-07 00:00:00,2021-01-29,7560.41,Delayed
...,...,...,...,...,...,...,...,...
95,5886,4462,106,2005-06-09,NaN,2005-06-29,6054.80,Out For Delivery
96,3659,3070,919,1986-03-26,NaN,1986-04-10,327.18,Out For Delivery
97,6969,4524,218,1989-01-15,1989-02-06 00:00:00,1989-01-29,515.14,Delayed
98,9165,4066,345,2016-08-29,2016-09-10 00:00:00,2016-09-18,4.24,Closed


In [566]:
orders_int_ship_id = All_dfs["internalorders"].IntShip_id	
internal_ship_id = All_dfs["internalshipments"].IntShip_id
ship_id_in_orders_ship_id = internal_ship_id.isin(orders_int_ship_id)
ship_id_not_in_orders_ship_id = list(All_dfs["internalshipments"].IntShip_id[[not elem for elem in ship_id_in_orders_ship_id]].unique())
len(ship_id_not_in_orders_ship_id)

522

In [567]:
c = 9931
for id in ship_id_not_in_orders_ship_id:
    new_row = [{"IntOrders_id":c, "IntShip_id":id, "quantity": 766, "placed_when": "1993-05-08",
    "actual_date":"1993-05-30 00:00:00", "expected_date":"1993-06-01", "cost":"38.90", "status":"Closed"}] 
    df = pd.DataFrame(new_row)
    c = c+1
    All_dfs["internalorders"] = pd.concat([All_dfs["internalorders"], df], ignore_index=True)


In [568]:
All_dfs["internalorders"]

,IntOrders_id,IntShip_id,quantity,placed_when,actual_date,expected_date,cost,status
0,8605,2360,766,1993-05-08,1993-05-30 00:00:00,1993-06-01,38.9,Closed
1,6266,6224,438,1975-01-08,NaN,1975-02-01,7.95,Out For Delivery
2,3057,1388,771,2007-10-29,2007-11-08 00:00:00,2007-11-16,8.24,Closed
3,7340,9456,870,2000-07-24,2000-08-07 00:00:00,2000-08-03,17.14,Delayed
4,5318,6798,801,2021-01-18,2021-02-07 00:00:00,2021-01-29,7560.41,Delayed
...,...,...,...,...,...,...,...,...
617,10448,10478,766,1993-05-08,1993-05-30 00:00:00,1993-06-01,38.90,Closed
618,10449,10479,766,1993-05-08,1993-05-30 00:00:00,1993-06-01,38.90,Closed
619,10450,10480,766,1993-05-08,1993-05-30 00:00:00,1993-06-01,38.90,Closed
620,10451,10481,766,1993-05-08,1993-05-30 00:00:00,1993-06-01,38.90,Closed


### Splitting Shipments tables

In [569]:
SRIntShip = All_dfs["internalshipments"].query('from_to_where == "SR"')
SRIntShip = SRIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SRIntShip
RCExtShip = All_dfs["externalshipments"].query('from_to_where == "RC"')
RCExtShip = RCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#RCExtShip
SSIntShip = All_dfs["internalshipments"].query('from_to_where == "SS"')
SSIntShip = SSIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SSIntShip
SCExtShip = All_dfs["externalshipments"].query('from_to_where == "SC"')
SCExtShip = SCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SCExtShip
All_dfs["RCExtShip".lower()] = RCExtShip
All_dfs["SCExtShip".lower()] = SCExtShip
All_dfs["SRIntShip".lower()] = SRIntShip
All_dfs["SSIntShip".lower()] = SSIntShip

### Splitting Orders Tables

In [570]:
filter_list = All_dfs["rcextship"]["ExtShip_id"]
All_dfs["rcextorders"] = All_dfs["externalorders"][All_dfs["externalorders"].ExtShip_id.isin(filter_list)].reset_index(drop=True)
filter_list = All_dfs["scextship"]["ExtShip_id"]
All_dfs["scextorders"] = All_dfs["externalorders"][All_dfs["externalorders"].ExtShip_id.isin(filter_list)].reset_index(drop=True)
#All_dfs["scextorders"] = All_dfs["externalorders"].query("ExtShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["srintship"]["IntShip_id"]
All_dfs["srintorders"] = All_dfs["internalorders"][All_dfs["internalorders"].IntShip_id.isin(filter_list)].reset_index(drop=True)
#All_dfs["srintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["ssintship"]["IntShip_id"]
All_dfs["ssintorders"] = All_dfs["internalorders"][All_dfs["internalorders"].IntShip_id.isin(filter_list)].reset_index(drop=True)
#All_dfs["ssintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)

### Removing Redundant Tables

In [571]:
All_dfs.pop("internalshipments")
All_dfs.pop("externalshipments")
All_dfs.pop("externalorders")
All_dfs.pop("internalorders")
All_dfs.keys()

dict_keys(['internaltransactions', 'facilities', 'internalservices', 'warehouses', 'customer', 'products', 'supplier', 'externaltransactions', 'manufacturing', 'externalservices', 'retailer', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

In [572]:
# for table in All_dfs:
#     print(table)
#     print(All_dfs[table].isna().sum())
#     print("_________________________")

### Removing nan

In [573]:
for table in All_dfs:
    All_dfs[table].fillna('Unknown',inplace = True)


### Execlude Tables

In [574]:
# All_dfs = {key: value for key, value in All_dfs.items() 
#            if key not in ["products","warehouses","manufacturing"]}
# All_dfs.keys()



### Initialize fk and ref_in

In [575]:
table_name = list(All_dfs.keys())
for table_name in All_dfs:
    fk[table_name] = {}
    ref_in[table_name] = set()

### Get Primary key for each table

In [576]:
for t in All_dfs:
    pk_finder(t,All_dfs[t])
pk

{'internaltransactions': 'IntTran_id',
 'facilities': 'fac_id',
 'internalservices': 'IntServ_id',
 'warehouses': 'warehouse_id',
 'customer': 'cust_id',
 'products': 'prod_id',
 'supplier': 'supp_id',
 'externaltransactions': 'ExtTran_id',
 'manufacturing': 'Manf_id',
 'externalservices': 'ExtServ_id',
 'retailer': 'retailer_id',
 'rcextship': 'ExtShip_id',
 'scextship': 'ExtShip_id',
 'srintship': 'IntShip_id',
 'ssintship': 'IntShip_id',
 'rcextorders': 'ExtOrders_id',
 'scextorders': 'ExtOrders_id',
 'srintorders': 'IntOrders_id',
 'ssintorders': 'IntOrders_id'}

In [577]:
ware_house_id = All_dfs["warehouses"].warehouse_id 

warehouse_product_isin_warehouse = All_dfs["products"].warehouses.explode().isin(ware_house_id)
warehouse_product_not_in_warehouse_values = list(All_dfs["products"].warehouses.explode()[[not elem for elem in warehouse_product_isin_warehouse]].unique())
warehouse_product_not_in_warehouse_values

[]

In [578]:
warehouse_isin_warehouse_product =  ware_house_id.isin(All_dfs["products"].warehouses.explode())
warehouses_to_be_replaced = list(ware_house_id[[not elem for elem in warehouse_isin_warehouse_product]].unique())[:len(warehouse_product_not_in_warehouse_values)]
warehouses_to_be_replaced

[]

### Get Foreing key for each table

In [579]:
x = All_dfs["warehouses"].warehouse_id.value_counts()
len(x[x>1])

0

In [580]:
All_dfs["products"].warehouses[0]

[5498, 6164, 5623, 7418, 8962, 7721, 4208, 8714, 1285]

In [581]:
All_dfs["products"].warehouses[0][0]

5498

In [582]:
for t in All_dfs:
    fk_finder(t,All_dfs[t])
fk

{'internaltransactions': {'CompFrom': 'supplier', 'Compto': 'supplier'},
 'facilities': {'supplier_id': 'supplier'},
 'internalservices': {'IntTrans_id': 'internaltransactions'},
 'warehouses': {},
 'customer': {},
 'products': {'warehouses': 'warehouses'},
 'supplier': {},
 'externaltransactions': {'CompFrom': 'supplier', 'Custto': 'customer'},
 'manufacturing': {'Different_suppliers': 'supplier',
  'Product_id': 'products',
  'Factory_id': 'supplier'},
 'externalservices': {'ExtTrans_id': 'externaltransactions'},
 'retailer': {},
 'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'scextship': {'factoryIds/retailerIds': 'supplier', 'idsTo': 'customer'},
 'srintship': {'listSuppIds': 'supplier', 'factoryIds': 'retailer'},
 'ssintship': {'listSuppIds': 'supplier', 'factoryIds': 'supplier'},
 'rcextorders': {'ExtShip_id': 'rcextship'},
 'scextorders': {'ExtShip_id': 'scextship'},
 'srintorders': {'IntShip_id': 'srintship'},
 'ssintorders': {'IntShip_id': 'ssintsh

In [583]:
asdadf

NameError: name 'asdadf' is not defined

In [ ]:
ref_in

### Determine nodes and edges

In [ ]:
nodes_edges(fk)

In [ ]:
nodes.keys()

In [ ]:
edges

In [ ]:
properties

### Create NodesTable

In [ ]:
nodesTable = add_nodes(nodes,nodesTable)
nodesTable

### Create Edges Table

In [ ]:
edgesTable = add_edges(edges, edgesTable,pk,fk,ref_in)
edgesTable

In [ ]:
# nodes_df = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
# edges_df = pd.DataFrame(columns=['From','To'])
all_nodes = list(nodes.keys())
for n in range(len(nodes)):
      for index, row in nodes[all_nodes[n]].iterrows():
          column_names = list(nodes[all_nodes[n]].columns) #get column names
          att = {}
          for i in range(1, len(column_names)):
              att[column_names[i]] = nodes[all_nodes[n]].iloc[index, i]
          newRow = [{'Label': all_nodes[n] , 'ID': nodes[all_nodes[n]].iloc[index,0] , 'Attributes': att}]
          tmp = pd.DataFrame(newRow)
          nodes_df = pd.concat([nodes_df, tmp], ignore_index=True)

In [ ]:
all_edges = list(edges.keys())
for n in range(len(edges)):
      Edge_name = all_edges[n]

      foreign_keys = list(fk[Edge_name].keys())

      from_col = foreign_keys[0]
      from_table_name = fk[Edge_name][from_col]
      from_df = All_dfs[from_table_name]
      from_df_pk = pk[from_table_name]

      to_col = foreign_keys[-1]
      to_table_name = fk[Edge_name][to_col]
      to_df = All_dfs[to_table_name]
      to_df_pk = pk[to_table_name]

      column_names = list(All_dfs[Edge_name].columns) #get column names
      for index, _ in All_dfs[Edge_name].iterrows():
          att = {}
          from_ref_id,to_ref_id = None, None


          for i in range(1, len(column_names)):
            column_name = column_names[i]


            if column_name not in foreign_keys:
              att[column_name] = All_dfs[Edge_name].iloc[index, i]

            else:
              reference_id = All_dfs[Edge_name].iloc[index, i]
              if column_name == from_col:
                # from_ref_id = from_df[from_df[from_df_pk] == reference_id].index[0]
                from_ref_id = reference_id
              else:
                # to_ref_id = to_df[to_df[to_df_pk] == reference_id].index[0]
                to_ref_id = reference_id

          # Adding new entry to node tabel
          newRow = [{'Label': Edge_name , 'ID': All_dfs[Edge_name].iloc[index,0] , 'Attributes': att}]
          tmp = pd.DataFrame(newRow)
          nodes_df = pd.concat([nodes_df, tmp], ignore_index=True)
          edge_node_index = len(nodes_df)-1
          # print(nodes_df.iloc[len(nodes_df)-1],All_dfs[Edge_name].iloc[index,0] )

          # creating two edges, one from the from_node to the edge node and one from edge node to to_node
          from_node_id = nodes_df[(nodes_df['Label']==from_table_name) &  (nodes_df['ID']==from_ref_id)].index[0]
          to_node_id =  nodes_df[(nodes_df['Label']==to_table_name) &  (nodes_df['ID']==to_ref_id)].index[0] 

          # from ---> edge 
          new_from_edge_row = [{'From': from_node_id , 'To':edge_node_index,'From_Table': from_table_name.capitalize(),'To_Table': Edge_name.capitalize() }]
          tmp = pd.DataFrame(new_from_edge_row)
          edges_df = pd.concat([edges_df, tmp], ignore_index=True)
          # edge --->to
          new_to_edge_row = [{'From': edge_node_index , 'To':  to_node_id, 'From_Table':Edge_name.capitalize(), 'To_Table': to_table_name.capitalize()}]
          tmp = pd.DataFrame(new_to_edge_row)
          edges_df = pd.concat([edges_df, tmp], ignore_index=True)



In [ ]:
all_properties = list(properties.keys())
for property_name in all_properties:
  property_df = All_dfs[property_name]
      
  foreign_keys = list(fk[property_name].keys())

  fk_col = foreign_keys[0]
  referenced_table_name = fk[property_name][fk_col]
  referenced_table_df = All_dfs[referenced_table_name]
  referenced_table_pk = pk[referenced_table_name]
  
  column_names = list(property_df.columns) #get column names
  
  for index, _ in property_df.iterrows():    
      att = {}
      reference_id = None;      
      
      for i in range(1, len(column_names)):
        column_name = column_names[i]
        
        
        if column_name not in foreign_keys:
          att[column_name] = property_df.iloc[index, i]
      
        else:
          # capturing foreign key value
          reference_id = property_df.iloc[index, i]

      # Adding new entry to node tabel
      newRow = [{'Label': property_name , 'ID': property_df.iloc[index,0] , 'Attributes': att}]
      tmp = pd.DataFrame(newRow)
      nodes_df = pd.concat([nodes_df, tmp], ignore_index=True)
      property_node_index = len(nodes_df)-1
      # print(nodes_df.iloc[len(nodes_df)-1],property_df.iloc[index,0] )

      # creating two edges, one from the from_node to the edge node and one from edge node to to_node
      referenced_node_id = nodes_df[(nodes_df['Label']==referenced_table_name) &  (nodes_df['ID']==reference_id)].index[0] 
      
      new_property_edge_row = [{'From': referenced_node_id , 'To':  property_node_index,'From_Table':referenced_table_name.capitalize(), 'To_Table': property_name.capitalize()}]
      tmp = pd.DataFrame(new_property_edge_row)
      edges_df = pd.concat([edges_df, tmp], ignore_index=True)



In [ ]:
nodes_df

In [ ]:
edges_df

In [ ]:
draw_graph_pyvis(nodes_df,edges_df)

[Neo 4j Function](#Neo4jFunction)


<a id="Neo4jCall"></a>

In [ ]:
# draw_graph_Neo4j(nodes_df,edges_df)

In [ ]:
sum_nodes = 0
sum_edges = 0
for key in All_dfs:
    table_size = len(All_dfs[key])
    print(key, table_size)
    sum_nodes += table_size
    if(key in all_edges):
        sum_edges += table_size*2
    if(key in all_properties ):
        sum_edges += table_size
print("Number Of Nodes: ",sum_nodes)
print("Number Of Edges: ",sum_edges)

In [ ]:
All_dfs["ssintorders"]

,IntOrders_id,IntShip_id,quantity,placed_when,actual_date,expected_date,cost,status
0,6266,6224,438,1975-01-08,Unknown,1975-02-01,7.95,Out For Delivery
1,5318,6798,801,2021-01-18,2021-02-07 00:00:00,2021-01-29,7560.41,Delayed
2,2439,8796,882,1978-07-09,Unknown,1978-07-28,35.13,Out For Delivery
3,1851,4895,220,2002-12-28,Unknown,2003-01-22,2.96,Out For Delivery
4,9800,3996,399,1975-10-06,Unknown,1975-10-29,83.16,Out For Delivery
5,5348,3541,809,1978-12-31,1979-01-30 00:00:00,1979-01-24,6.34,Delayed
6,4034,5161,347,1998-12-26,Unknown,1999-01-06,4611.72,Out For Delivery
7,4939,5797,819,1970-01-23,Unknown,1970-02-18,6.82,Out For Delivery
8,5668,8796,232,1997-06-22,1997-07-20 00:00:00,1997-07-16,7.89,Delayed
9,8787,6627,572,2009-02-14,2009-03-09 00:00:00,2009-03-02,9.90,Delayed


In [ ]:
All_dfs["ssintship"]

,IntShip_id,listSuppIds,factoryIds
0,7281,90942,52160
1,3477,87916,30692
2,8796,90236,26533
3,2933,76443,51293
4,3000,14125,68357
...,...,...,...
532,10478,42179,46070
533,10479,48467,46070
534,10480,70468,18952
535,10481,48580,18952


In [ ]:
ware_house_id = All_dfs["ssintship"].IntShip_id	

warehouse_product_isin_warehouse = All_dfs["ssintorders"].IntShip_id.isin(ware_house_id)
warehouse_product_not_in_warehouse_values = list(All_dfs["ssintorders"].IntShip_id[[not elem for elem in warehouse_product_isin_warehouse]].unique())
warehouse_product_not_in_warehouse_values

[]

In [ ]:
orders_int_ship_id = All_dfs["ssintorders"].IntShip_id	
internal_ship_id = All_dfs["ssintship"].IntShip_id
ship_id_in_orders_ship_id = internal_ship_id.isin(orders_int_ship_id)
ship_id_not_in_orders_ship_id = list(All_dfs["ssintship"].IntShip_id[[not elem for elem in ship_id_in_orders_ship_id]].unique())
ship_id_not_in_orders_ship_id

[3000,
 9411,
 7182,
 1983,
 4227,
 2748,
 9470,
 2035,
 1945,
 8258,
 4906,
 2906,
 8070,
 2199,
 9995,
 9996,
 9997,
 9998,
 9999,
 10000,
 10001,
 10002,
 10003,
 10004,
 10005,
 10006,
 10007,
 10008,
 10009,
 10010,
 10011,
 10012,
 10013,
 10014,
 10015,
 10016,
 10017,
 10018,
 10019,
 10020,
 10021,
 10022,
 10023,
 10024,
 10025,
 10026,
 10027,
 10028,
 10029,
 10030,
 10031,
 10032,
 10033,
 10034,
 10035,
 10036,
 10037,
 10038,
 10039,
 10040,
 10041,
 10042,
 10043,
 10044,
 10045,
 10046,
 10047,
 10048,
 10049,
 10050,
 10051,
 10052,
 10053,
 10054,
 10055,
 10056,
 10057,
 10058,
 10059,
 10060,
 10061,
 10062,
 10063,
 10064,
 10065,
 10066,
 10067,
 10068,
 10069,
 10070,
 10071,
 10072,
 10073,
 10074,
 10075,
 10076,
 10077,
 10078,
 10079,
 10080,
 10081,
 10082,
 10083,
 10084,
 10085,
 10086,
 10087,
 10088,
 10089,
 10090,
 10091,
 10092,
 10093,
 10094,
 10095,
 10096,
 10097,
 10098,
 10099,
 10100,
 10101,
 10102,
 10103,
 10104,
 10105,
 10106,
 10107,
 10

In [ ]:
c = 9931
for id in ship_id_not_in_orders_ship_id:
    new_row = [{"IntOrders_id":c, "IntShip_id":id, "quantity": 766, "placed_when": "1993-05-08",
    "actual_date":"1993-05-30 00:00:00", "expected_date":"1993-06-01", "cost":"38.90", "status":"Closed"}] 
    df = pd.DataFrame(new_row)
    c = c+1
    All_dfs["internalorders"] = pd.concat([All_dfs["internalorders"], df], ignore_index=True)
